# gene location comp

In [4]:
from scprint.tokenizers import protein_embeddings_generator
from scprint.utils.get_seq import load_fasta_species
%reload_ext autoreload
%autoreload 2

from scprint.utils.utils import run_command
import pandas as pd
from Bio import SeqIO


→ connected lamindb: jkobject/scprint2


In [3]:
names = {
    #"homo_sapiens": "NCBITaxon:9606",
    "mus_musculus": "NCBITaxon:10090",
    "arabidopsis_thaliana": "NCBITaxon:3702",
    "bos_taurus": "NCBITaxon:9913",
    "caenorhabditis_elegans": "NCBITaxon:6239",
    "callithrix_jacchus": "NCBITaxon:9483",
    "danio_rerio": "NCBITaxon:7955",
    "drosophila_melanogaster": "NCBITaxon:7227",
    "gallus_gallus": "NCBITaxon:9031",
    "heterocephalus_glaber_male": "NCBITaxon:10181",
    "macaca_mulatta": "NCBITaxon:9544",
    "oryctolagus_cuniculus": "NCBITaxon:9986",
    #"oryza_sativa": "NCBITaxon:39947",
    "ovis_aries": "NCBITaxon:9940",
    "pan_troglodytes": "NCBITaxon:9598",
    "sus_scrofa": "NCBITaxon:9823",
    "zea_mays": "NCBITaxon:4577"
}

In [6]:
def get_locs(file):
    locs= []
    dup = set()
    noloc = 0
    for record in SeqIO.parse(file[:-3], "fasta"):
        name = record.description.split(' gene:')[1].split(' ')[0].split('.')[0]
        if "chromosome:" in record.description:
            val = record.description.split(' chromosome:')[1].split(' ')[0]
            ref, chrom, start, end, _ = val.split(':')
        elif "primary_assembly" in record.description:
            val = record.description.split(' primary_assembly:')[1].split(' ')[0]
            ref, chrom, start, end, _ = val.split(':')
        else:
            noloc += 1
            continue
        if name in dup:
            continue
        dup.add(name)
        locs.append([name, chrom, start, end])
    print(len(dup), " genes had duplicates")
    print(noloc, " genes had no location")
    df = pd.DataFrame(locs, columns=['name', 'chrom', 'start', 'end'])
    df = df.astype({'start': 'int32', 'end': 'int32'})
    df = df.sort_values(by=['chrom', 'start']).reset_index(drop=True)
    return df


# test on humans

In [4]:
file = load_fasta_species(species="homo_sapiens", load=["cdna"])

In [5]:
file = file[0]
fname = file.split("/")[-1]
run_command(["gunzip", file])

gzip: /tmp/data/fasta/Homo_sapiens.GRCh38.cdna.all.fa already exists;	not overwritten


2

In [ ]:
df = get_locs(file)


In [9]:
import os
os.makedirs("../data/main/gene_locs", exist_ok=True)

In [13]:
df.set_index("name").to_parquet("../data/main/gene_locs/homo.parquet")

In [7]:
for species in names.keys():
    file = load_fasta_species(species=species, load=["cdna"])
    file = file[0]
    fname = file.split("/")[-1]
    run_command(["gunzip", file])
    df = get_locs(file)
    df.set_index("name").to_parquet(f"../data/main/gene_locs/{species.split('_')[0]}.parquet")

gzip: /tmp/data/fasta/Mus_musculus.GRCm39.cdna.all.fa already exists;	not overwritten


35937  genes had duplicates
54  genes had no location


gzip: /tmp/data/fasta/Arabidopsis_thaliana.TAIR10.cdna.all.fa already exists;	not overwritten


27655  genes had duplicates
0  genes had no location


gzip: /tmp/data/fasta/Bos_taurus.ARS-UCD1.2.cdna.all.fa already exists;	not overwritten


...

22505  genes had duplicates
2393  genes had no location


gzip: /tmp/data/fasta/Sus_scrofa.Sscrofa11.1.cdna.all.fa already exists;	not overwritten


22516  genes had duplicates
0  genes had no location


gzip: /tmp/data/fasta/Zea_mays.Zm-B73-REFERENCE-NAM-5.0.cdna.all.fa already exists;	not overwritten


39035  genes had duplicates
748  genes had no location


In [ ]:
get_locs("/tmp/data/fasta/Bos_taurus.ARS-UCD1.2.cdna.all.fa.gz")

In [29]:
list(names.keys())[2]

'bos_taurus'

In [13]:
pd.read_parquet(f"../data/main/gene_locs/{list(names.keys())[0].split('_')[0]}.parquet")

,chrom,start,end
name,,,
ENSMUSG00000051951,1,3276124,3286567
ENSMUSG00000102851,1,3322980,3323459
ENSMUSG00000103147,1,3602018,3602943
ENSMUSG00000025900,1,4069780,4479464
ENSMUSG00000102948,1,4326457,4330742
...,...,...,...
ENSMUSG00000100246,Y,90010986,90011680
ENSMUSG00000096178,Y,90412517,90444532
ENSMUSG00000100964,Y,90510316,90542914


In [15]:
ls ../data/main/gene_locs/

arabidopsis.parquet     gallus.parquet          ovis.parquet
bos.parquet             heterocephalus.parquet  pan.parquet
caenorhabditis.parquet  homo.parquet            sus.parquet
callithrix.parquet      macaca.parquet          zea.parquet
danio.parquet           mus.parquet
drosophila.parquet      oryctolagus.parquet


In [17]:
pd.read_parquet(f"../data/main/gene_locs/")

,chrom,start,end
name,,,
AT1G01010,1,3631,5899
AT1G01020,1,6788,9130
AT1G01030,1,11649,13714
AT1G01040,1,23121,31227
AT1G01050,1,31382,33009
...,...,...,...
Zm00001eb404690,9,162278225,162280767
Zm00001eb404700,9,162498899,162511418
Zm00001eb404710,9,162552801,162575880
